In [1]:
!python --version # should be 3.6

Python 3.6.3


In [2]:
from deeppavlov import configs, build_model
import nltk
import pandas as pd


import tqdm

from fuzzywuzzy import fuzz

from natasha import (
    DatesExtractor,
    MoneyExtractor,
    AddrExtractor,
    MorphVocab,
)

from natasha.obj import Money
from natasha.obj import Addr
from natasha.obj import Date
from natasha.obj import AddrPart

from natasha import (
    NewsNERTagger,
    NewsEmbedding,
)
emb = NewsEmbedding()
ner = NewsNERTagger(emb)

morph_vocab = MorphVocab()
extractors = [
    AddrExtractor(morph_vocab),
    DatesExtractor(morph_vocab),
    MoneyExtractor(morph_vocab)
]

# !python3 -m deeppavlov install ner_ontonotes_bert_mult

/home/droman/Documents/diploma/deeppavlov_ner_3.6/venv/lib/python3.6/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
config_path = configs.ner.ner_rus_bert
ner = build_model(config_path, download=False)
ner_multi = build_model(configs.ner.ner_ontonotes_bert_mult, download=False)

[nltk_data] Downloading package punkt to /home/droman/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/droman/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to
[nltk_data]     /home/droman/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /home/droman/nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!


2021-04-18 10:05:04.70 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /home/droman/.deeppavlov/models/ner_rus_bert/tag.dict]











The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API



Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


Instructions for updating:
Use stan

2021-04-18 10:05:19.263 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /home/droman/.deeppavlov/models/ner_rus_bert/model]



INFO:tensorflow:Restoring parameters from /home/droman/.deeppavlov/models/ner_rus_bert/model


2021-04-18 10:05:23.110 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /home/droman/.deeppavlov/models/ner_ontonotes_bert_mult/tag.dict]


2021-04-18 10:05:42.912 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /home/droman/.deeppavlov/models/ner_ontonotes_bert_mult/model]


INFO:tensorflow:Restoring parameters from /home/droman/.deeppavlov/models/ner_ontonotes_bert_mult/model


In [ ]:
# sentence = 'Директором ОАО "Страна Чудес" назначена Алиса'
#
# # Можно так же список слов
# # sentence = ['Директором', 'ОАО', '"', 'Страна', 'Чудес', '"', 'назначена', 'Алиса']
#
# tokens, tags = ner(['Ио́сиф Виссарио́нович Ста́лин — российский революционер, советский политический, государственный, военный и партийный деятель. С 21 января 1924 по 5 марта 1953 — руководитель СССР. Маршал Советского Союза, Генералиссимус Советского Союза.'])
# for tok, tag in zip(tokens[0], tags[0]):
#     print(f'{tok}\t{tag}')
#
# example_str = ['Ио́сиф Виссарио́нович Ста́лин — российский революционер, советский политический, государственный, военный и партийный деятель. С 21 января 1924 по 5 марта 1953 — руководитель СССР. Маршал Советского Союза, Генералиссимус Советского Союза.']
#

In [3]:
def print_tokens(tokens, tags):
    for tok, tag in zip(tokens, tags):
        print(f'{tok}\t{tag}')

def filter_after_ner(tokens, tags):
    stop_words = ['́']
    excluded_indexes = []
    for i in range(len(tokens)):
        if tokens[i] in stop_words:
            excluded_indexes.append(i)
    def filter_by_index(l):
        return [l[i] for i in range(len(l)) if i not in excluded_indexes]
    return filter_by_index(tokens), filter_by_index(tags)

def get_deeppavlov_entities(tokens, tags):
    entities = []
    entity = ''
    for tok, tag in zip(tokens, tags):
        if tag == 'O' and entity != '':
            entities.append(entity)
            entity = ''
        if 'B-' in tag:
            entity = tok
        if 'I-' in tag:
            entity += ' ' + tok
    return entities

def nearest(sent1, sent2):
    for i in sent2:
        distance = fuzz.token_sort_ratio(sent1.lower(), i.lower()) #nltk.edit_distance(sent1, i)
        if distance >= 80:
            return True
        # if (distance/len(sent1)) * 100 <= 30:
        #     return True
    return False

def get_percent(part_entities, full_entities):
    max_count = len(full_entities)
    real_count = 0
    for i in part_entities:
        if nearest(i, full_entities):
            real_count += 1
    if max_count == 0:
        return 0.
    return real_count / max_count

def get_entities_from_natasha_extractors(text):
    spans = []
    entities = []
    for extractor in extractors:
        matches = extractor(text)
        spans.extend(_ for _ in matches)
    for span in spans:
        fact = span.fact
        if type(fact) is Money:
            entities.append(str(fact.amount))
        else:
            if type(fact) is Date:
                entities.append(str(fact.year) + " " + str(fact.month) + " " + str(fact.day))
            else:
                if type(fact) is AddrPart and fact.type is not None:
                    entities.append(str(fact.value))
    return entities

def get_natasha_ner_entities(text):
    markup = ner(text)
    spans = markup.spans
    entities = []
    for span in spans:
        entities.append(text[span.start: span.stop])
    return entities

def get_natasha_entities(text):
    return list(set(get_entities_from_natasha_extractors(text) + get_natasha_ner_entities(text)))

def get_overlap_entities_percent(sent1, sent2, model_ents_funcs):
    entities1 = set()
    entities2 = set()

    for model in model_ents_funcs:
        entities1.update(model(sent1))
        entities2.update(model(sent2))

    return get_percent(entities1, entities2)

def get_deeppavlov_ents(sent):
    tokens1, tags1 = ner_multi(sent)
    tokens1, tags1 = tokens1[0], tags1[0]

    filtered_tokens1, filtered_tags1 = filter_after_ner(tokens1, tags1)

    return get_deeppavlov_entities(filtered_tokens1, filtered_tags1)

def get_ner_overlap_feature(data):
    original = data["original"].tolist()
    targets = data["scored_text"].tolist()
    feature_values = []
    ner_ents_extractors = [get_deeppavlov_ents, get_natasha_entities]

    for i in tqdm.tqdm(range(len(original))):
        origin = original[i]
        target = targets[i]
        try:
            if target == " " or target == "" or target is None:
                feature_values.append(0.)
                continue
            feature_value = get_overlap_entities_percent([target], [origin], ner_ents_extractors)
            feature_values.append(feature_value)
        except:
            feature_values.append(0.)
            print("index: ", i)
            print("raget text: ", target)
            print("origin text: ", origin)
    return feature_values